# 4-Oscar Prediction with AutoML
After out dataframe has been assemlbed (see scraping and table_assembling) notebooks we have the data we need to make predictions on the Best Picture winner. [AutoML](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html) represents a quick, but powerful route though the Machine Learning process. H2O's AutoML runs many models through the dataset and using cross-validation, picks the best one. For my purposes I use it to confirm/compare to the Preferential Balloting Random Forest model I created.
If you are gunning to win your office's Oscar pool, scroll down to see the results.

In [1]:
import pandas as pd
import numpy as np
import h2o

# Machine Learning - Using h2o Auto ML

In [2]:
full_table = pd.read_csv('../data/processed_results/extended_df.csv')
full_table=full_table.drop('Unnamed: 0', axis=1)
full_table

,year,film,wiki,winner,rating,numVotes,worldwide_box_office,Action,Adventure,Animation,...,nom_pga,winner_pga,nom_bafta,winner_bafta,nom_dga,winner_dga,nom_sag,winner_sag,nom_cannes,winner_cannes
0,1927,Wings,/wiki/Wings_(1927_film),True,7.3,13576.0,$746,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1927,7th Heaven,/wiki/7th_Heaven_(1927_film),False,5.2,26223.0,"$79,808",0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1927,The Racket,/wiki/The_Racket_(1928_film),False,6.7,3149.0,"$21,733,230",0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1928,The Broadway Melody,/wiki/The_Broadway_Melody,True,5.6,7605.0,"$223,723",0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1928,Alibi,/wiki/Alibi_(1929_film),False,7.4,391.0,"$42,915",0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,2022,The Fabelmans,/wiki/The_Fabelmans,False,7.6,85709.0,"$45,164,110",0,0,0,...,1,0,0,0,1,0,1,0,0,0
547,2022,Tár,/wiki/T%C3%A1r,False,7.5,69684.0,"$27,541,681",0,0,0,...,1,0,1,0,1,0,0,0,0,0
548,2022,Top Gun: Maverick,/wiki/Top_Gun:_Maverick,False,8.3,577408.0,"$1,493,491,858",0,0,0,...,1,0,0,0,1,0,0,0,0,0
549,2022,Triangle of Sadness,/wiki/Triangle_of_Sadness,False,7.3,128812.0,"$25,615,870",0,0,0,...,0,0,0,0,0,0,0,0,1,1


In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.371-b11, mixed mode)
  Starting server from C:\Users\aczaplak.MPD1\AppData\Local\Programs\Python\Python310\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\ACZAPL~1.MPD\AppData\Local\Temp\tmp11dqmq52
  JVM stdout: C:\Users\ACZAPL~1.MPD\AppData\Local\Temp\tmp11dqmq52\h2o_aczaplak_started_from_python.out
  JVM stderr: C:\Users\ACZAPL~1.MPD\AppData\Local\Temp\tmp11dqmq52\h2o_aczaplak_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,08 secs
H2O_cluster_timezone:,Europe/Belgrade
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.4
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_aczaplak_wkb9zy
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.752 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


First Year of Existance. This data will be used below
- golden_globes 1943
- pga 1989
- bafta 1960
- dga 1948
- sag 1995
- cannes 1970

In [4]:
# I pick a min_year where the awards shows will be relevant
min_year = 1995

# H2O's Auto ML

In [5]:
# Auto ML uses Cross Validation, so we do not specifiy a validation set
train = full_table.loc[((full_table['year'] < 2022) & (full_table['year'] >= min_year))]

print('training set contains:', train.shape[0], 'movies')

training set contains: 184 movies


In [8]:
#train = train.drop(['index', '[]'], axis=1)
train.columns

Index(['year', 'film', 'wiki', 'winner', 'rating', 'numVotes',
       'worldwide_box_office', 'Action', 'Adventure', 'Animation', 'Biography',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy',
       'Film-Noir', 'Game-Show', 'History', 'Horror', 'Music', 'Musical',
       'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport',
       'Talk-Show', 'Thriller', 'War', 'Western', 'action', 'adventure',
       'animation', 'biography', 'comedy', 'crime', 'documentary', 'drama',
       'family', 'fantasy', 'film-noir', 'history', 'horror', 'music',
       'musical', 'mystery', 'romance', 'sci-fi', 'sport', 'thriller', 'war',
       'western', 'nominations', 'Oscar_win', 'nom_gg_drama',
       'winner_gg_drama', 'nom_gg_comedy', 'winner_gg_comedy', 'nom_pga',
       'winner_pga', 'nom_bafta', 'winner_bafta', 'nom_dga', 'winner_dga',
       'nom_sag', 'winner_sag', 'nom_cannes', 'winner_cannes'],
      dtype='object')

In [6]:
print(type(train))

<class 'pandas.core.frame.DataFrame'>


In [7]:
from h2o.automl import H2OAutoML, get_leaderboard

# Import a sample binary outcome train/test set into H2O
train1 = h2o.H2OFrame(train)

# Identify predictors and response
predictors = ['year', 'film', 'wiki', 'rating', 'numVotes',
       'worldwide_box_office', 'Action', 'Adventure', 'Animation', 'Biography',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy',
       'Film-Noir', 'Game-Show', 'History', 'Horror', 'Music', 'Musical',
       'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport',
       'Talk-Show', 'Thriller', 'War', 'Western', 'action', 'adventure',
       'animation', 'biography', 'comedy', 'crime', 'documentary', 'drama',
       'family', 'fantasy', 'film-noir', 'history', 'horror', 'music',
       'musical', 'mystery', 'romance', 'sci-fi', 'sport', 'thriller', 'war',
       'western', 'nominations', 'nom_gg_drama',
       'winner_gg_drama', 'nom_gg_comedy', 'winner_gg_comedy', 'nom_pga',
       'winner_pga', 'nom_bafta', 'winner_bafta', 'nom_dga', 'winner_dga',
       'nom_sag', 'winner_sag', 'nom_cannes', 'winner_cannes']

x = predictors
y = 'Oscar_win'

# For binary classification, response should be a factor
train1[y] = train1[y].asfactor()

# Run AutoML for 100 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=100, seed=1
                , keep_cross_validation_predictions= True
               , exclude_algos = ['StackedEnsemble'])

aml.train(x=x, y=y, training_frame=train1)

# AutoML Leaderboard
lb = aml.leaderboard

# Optionally edd extra model information to the leaderboard
lb = get_leaderboard(aml, extra_columns='ALL')

# Print all rows (instead of default 10 rows)
lb.head(rows=lb.nrows)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
16:08:32.296: AutoML: XGBoost is not available; skipping it.
16:08:32.571: _train param, Dropping bad and constant columns: [Film-Noir, Sport, Horror, winner_cannes, worldwide_box_office, wiki, nom_cannes, film, film-noir, documentary]

███
16:08:34.435: _train param, Dropping bad and constant columns: [Film-Noir, Sport, Horror, winner_cannes, worldwide_box_office, wiki, nom_cannes, film, film-noir, documentary]
16:08:34.435: _min_rows param, The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 184.0.
16:08:34.444: _train param, Dropping bad and constant columns: [Film-Noir, Sport, Horror, winner_cannes, worldwide_box_office, wiki, nom_cannes, film, film-noir, documentary]
16:08:36.827: _train param, Dropping bad and constant columns: [Film-Noir, Sport, Horror, winner_cannes, worldwide_box_office, wiki, nom_canne

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms,algo
DeepLearning_grid_2_AutoML_1_20230517_160832_model_4,0.727766,0.680794,0.29176,0.292876,0.395659,0.156546,5261,0.129583,DeepLearning
DeepLearning_grid_3_AutoML_1_20230517_160832_model_1,0.707714,0.717919,0.407089,0.361996,0.370955,0.137608,10546,0.102322,DeepLearning
DeepLearning_grid_1_AutoML_1_20230517_160832_model_23,0.687662,1.00964,0.257296,0.306558,0.491412,0.241486,3660,0.042359,DeepLearning
DeepLearning_grid_1_AutoML_1_20230517_160832_model_7,0.673036,0.711369,0.247073,0.319533,0.400961,0.16077,9201,0.053295,DeepLearning
DeepLearning_grid_2_AutoML_1_20230517_160832_model_14,0.664544,0.751402,0.234738,0.319297,0.38431,0.147694,3788,0.082997,DeepLearning
DeepLearning_grid_1_AutoML_1_20230517_160832_model_15,0.660415,1.94678,0.248945,0.342062,0.458766,0.210466,2643,0.043961,DeepLearning
DeepLearning_grid_2_AutoML_1_20230517_160832_model_16,0.64803,1.65277,0.333943,0.356806,0.395622,0.156517,2735,0.082518,DeepLearning
DeepLearning_grid_1_AutoML_1_20230517_160832_model_8,0.643548,0.712609,0.237042,0.374499,0.400227,0.160181,9149,0.070535,DeepLearning
DeepLearning_grid_3_AutoML_1_20230517_160832_model_19,0.63883,0.813923,0.185431,0.354565,0.403464,0.162783,5334,0.146382,DeepLearning
DeepLearning_grid_3_AutoML_1_20230517_160832_model_14,0.627978,0.575585,0.225945,0.373909,0.373781,0.139712,6699,0.067032,DeepLearning


In [8]:
top_model = aml.leader
top_model

Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_grid_2_AutoML_1_20230517_160832_model_4


Status of Neuron Layers: predicting Oscar_win, 2-class classification, bernoulli distribution, CrossEntropy loss, 16 402 weights/biases, 205,0 KB, 57 040 training samples, mini-batch size 1
    layer    units    type              dropout    l1    l2    mean_rate             rate_rms             momentum    mean_weight             weight_rms           mean_bias               bias_rms
--  -------  -------  ----------------  ---------  ----  ----  --------------------  -------------------  ----------  ----------------------  -------------------  ----------------------  --------------------
    1        60       Input             10.0
    2        100      RectifierDropout  0.0        0.0   0.0   0.02149445718031105   0.0326448529958725   0.0         0.0016446201505411107   0.11460211873054504  0.4955984703253772      0.0536341518163681
    3        100      RectifierDropout  0.0        0.0   0.0   0.024681739731851848  0.05916735529899597  0.0         -0.0018280174089241427  0.1061948835849762   0.9973499979376966      0.013858620077371597
    4        2        Softmax                      0.0   0.0   0.007501548861619085  0.01723358780145645  0.0         0.027626219384837895    0.571382999420166    -0.0001903927322003127  0.006368305534124374

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.0013157007635092753
RMSE: 0.03627258969951381
LogLoss: 0.008036088363151436
Mean Per-Class Error: 0.0
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6689686039884123
       0    1    Error    Rate
-----  ---  ---  -------  -----------
0      157  0    0        (0.0/157.0)
1      0    27   0        (0.0/27.0)
Total  157  27   0        (0.0/184.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.668969     1         24
max f2                       0.668969     1         24
max f0point5                 0.668969     1         24
max accuracy                 0.668969     1         24
max precision                1            1         0
max recall                   0.668969     1         24
max specificity              1            1         0
max absolute_mcc             0.668969     1         24
max min_per_class_accuracy   0.668969     1         24
max mean_per_class_accuracy  0.668969     1         24
max tns                      1            157       0
max fns                      1            26        0
max fps                      1.53053e-14  157       181
max tps                      0.668969     27        24
max tnr                      1            1         0
max fnr                      1            0.962963  0
max fpr                      1.53053e-14  1         181
max tpr                      0.668969     1         24

Gains/Lift Table: Avg response rate: 14,67 %, avg score: 14,82 %
group    cumulative_data_fraction    lower_threshold    lift     cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain     cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  -------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  -------  -----------------  --------------------
1        0.0108696                   1                  6.81481  6.81481            1                1            1                           1                   0.0740741       0.0740741                  581.481  581.481            0.0740741
2        0.0217391                   1                  6.81481  6.81481            1                1            1                           1                   0

## Predict the winner

In [13]:
# Predict on 2019's films
test = full_table.loc[(full_table['year'] == 2021)]

# Import a binary outcome train/test set into H2O
test = h2o.H2OFrame(test)

# For binary classification, response should be a factor
test[y] = test[y].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [14]:
preds = top_model.predict(test)

preds

deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%


predict,p0,p1
0,0.999976,2.37118e-05
1,0.00165605,0.998344
0,0.995174,0.00482619
0,1,1.74511e-07
0,0.995249,0.00475117
0,0.999972,2.78123e-05
0,0.999979,2.0885e-05
0,1,9.07248e-09
0,0.999955,4.5392e-05
0,0.999999,1.10776e-06


In [15]:
test['pred'] = preds['predict']
test['probA'] = preds['p1']
test_pd = test.as_data_frame(use_pandas=True)

In [16]:
final_rankings = test_pd[['film','probA']].sort_values('probA', ascending = False)
final_rankings['%_confidence'] = final_rankings['probA']/final_rankings['probA'].sum() * 100
final_rankings

,film,probA,%_confidence
1,CODA,9.983440e-01,9.903809e+01
2,Belfast,4.826187e-03,4.787692e-01
4,Drive My Car,4.751175e-03,4.713278e-01
8,Nightmare Alley,4.539204e-05,4.502998e-03
5,Dune,2.781235e-05,2.759051e-03
0,West Side Story,2.371181e-05,2.352268e-03
6,King Richard,2.088495e-05,2.071837e-03
9,The Power of the Dog,1.107759e-06,1.098923e-04
3,Don't Look Up,1.745112e-07,1.731192e-05
7,Licorice Pizza,9.072485e-09,9.000120e-07


# And the Oscar goes to...

In [13]:
bp_winner = np.array(final_rankings.reset_index())[0][1].split('(')[0].strip()
print(f'And the Oscar goes to...\n🎉🏆{bp_winner}🏆🎉')

And the Oscar goes to...
🎉🏆1917🏆🎉
